In [6]:
import torch
from torch import tensor, nn
from loading_datas import  generate_pair_sets
import torch.nn.functional as F


In [7]:
torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available()
                              else torch.FloatTensor)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


train_pairs, train_target, train_classes, test_pairs, test_target, test_classes = generate_pair_sets(1000)
train_pairs = train_pairs.to(device)
train_target = train_target.to(device)
train_classes = train_classes.to(device)
test_pairs = test_pairs.to(device)
test_target = test_target.to(device)
test_classes = test_classes.to(device)

cuda


In [81]:
def compute_nb_errors(model, data_input, data_target,batch_size, aux):

    nb_data_errors = 0

    for inputs, targets in zip(data_input.split(batch_size), data_target.split(batch_size)):
        if aux:
            output = model(inputs)[0]
        else:
            output = model(inputs)
            
        _, predicted_classes = torch.max(output.narrow(1,0,2), 1)
        for k in range(len(targets)):
            if targets[k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1        
    return nb_data_errors


In [82]:

def training_model(model,
                   train_input,train_target,train_classes,
                   test_input,test_target,
                   eta,epochs = 25,
                   batch_size = 50,
                   auxiliary_loss = True):
  
  # test_acc = [0] * epochs

  # define criterion and optimizer
  # need to check the other possibilities
  criterion = nn.CrossEntropyLoss()
  aux_critierion = nn.NLLLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr = eta)


  

  for e in range(0,epochs):

    for inputs,target,classes in zip(train_input.split(batch_size),train_target.split(batch_size),train_classes.split(batch_size)):


      if auxiliary_loss :
        outputs = model(inputs)

        prediction = outputs.narrow(1,0,2)
        aux_1 = outputs.narrow(1,2,10)
        aux_2 = outputs.narrow(1,12,10)

        loss = criterion(prediction, target) 
        loss_1 = aux_critierion(torch.sigmoid(aux_1), classes[:,0]) 
        loss_2 = aux_critierion(torch.sigmoid(aux_2), classes[:,1]) 
        # print(loss_1)

        loss += loss_1 + loss_2

      else : 
        outputs = model(inputs)
        loss = criterion(outputs, target)

      # optimising parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step() 

  
    # train_acc = 0
    train_acc = 100 * (1 - compute_nb_errors(model, train_input, train_target,batch_size, False)/train_input.size(0))
    test_acc = 100 * (1 - compute_nb_errors(model, test_input, test_target,batch_size, False)/test_input.size(0))
    print(f"Epoch # {e+1} / Train accuracy (%): {train_acc:.2f} / Test accuracy (%): {test_acc:.2f}")
     


  return test_acc

In [10]:
class Net1(nn.Module):
  def __init__(self):
    super().__init__()
    #channel 1
    self.conv11 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv12 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv13 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv14 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv15 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    #channel 2
    self.conv21 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv22 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv23 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv24 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv25 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    self.maxPool = nn.MaxPool2d(kernel_size=(2, 2))



    self.fc1 = nn.Linear(512*3*3, 512)
    self.fc2 = nn.Linear(512, 64)
    self.fc3 = nn.Linear(64, 2)

  def forward(self,x): 
    # spliting the channels
    c1 = torch.narrow(x,1,0,1)
    c2 = torch.narrow(x,1,1,1)

    # Channel 1
    c1 = F.relu(self.conv11(c1))  #16 x 14 x 14
    c1 = F.relu(self.conv12(c1))  # 32 x 13 x 13
    c1 = self.maxPool(c1)
    c1 = F.relu(self.conv13(c1))  # 64 x 7 x 7
    c1 = F.relu(self.conv14(c1))  # 128 x 6 x 6
    c1 = F.relu(self.conv15(c1))  # 256 x 6 x 6
    c1 = self.maxPool(c1)  # 256 x 3 x 3

    # Channel 2
    c2 = F.relu(self.conv21(c2))  # 16 x 14 x 14
    c2 = F.relu(self.conv22(c2))  # 32 x 13 x 13
    c2 = self.maxPool(c2)
    c2 = F.relu(self.conv23(c2))  # 64 x 7 x 7
    c2 = F.relu(self.conv24(c2))  # 128 x 6 x 6
    c2 = F.relu(self.conv25(c2))  # 256 x 6 x 6
    c2 = self.maxPool(c2)  # 256 x 3 x 3





    output = torch.cat((c1,c2),1)

    output = output.view(-1,512*3*3)

    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    output = self.fc3(output)

    return output



In [11]:
class Net1_aux(nn.Module):
  def __init__(self):
    super().__init__()
    #channel 1
    self.conv11 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv12 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv13 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv14 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv15 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    #channel 2
    self.conv21 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv22 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv23 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv24 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv25 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    self.maxPool = nn.MaxPool2d(kernel_size=(2, 2))



    self.fc1 = nn.Linear(512*3*3, 512)
    self.fc2 = nn.Linear(512, 64)
    self.fc3 = nn.Linear(64, 2)

  def aux(self, img1, img2):
    img = output = torch.cat((img1, img2), 1)
    nb_el = img.size()[1] * img.size()[2] * img.size()[3]
    # print(nb_el)
    output = img.view(-1, nb_el)
    # print(output.size())
    fc1_aux = nn.Linear(nb_el, 512)
    output = F.relu(fc1_aux(output))
    output = F.relu(self.fc2(output))
    output = self.fc3(output)
    return output

  def forward(self,x): 
    # spliting the channels
    c1 = torch.narrow(x,1,0,1)
    c2 = torch.narrow(x,1,1,1)

    # Channel 1
    c1 = F.relu(self.conv11(c1))  #16 x 14 x 14
    c2 = F.relu(self.conv21(c2))  # 16 x 14 x 14
    output1 = self.aux(c1, c2)


    # Channel 2
    c1 = F.relu(self.conv12(c1))  # 32 x 13 x 13
    c2 = F.relu(self.conv22(c2))  # 32 x 13 x 13

    
    c1 = self.maxPool(c1)
    c2 = self.maxPool(c2)
    output2 = self.aux(c1, c2)


    c1 = F.relu(self.conv13(c1))  # 64 x 7 x 7
    c2 = F.relu(self.conv23(c2))  # 64 x 7 x 7


    c1 = F.relu(self.conv14(c1))  # 128 x 6 x 6
    c2 = F.relu(self.conv24(c2))  # 128 x 6 x 6

    
    output3 = self.aux(c1, c2)

    c1 = F.relu(self.conv15(c1))  # 256 x 6 x 6
    c2 = F.relu(self.conv25(c2))  # 256 x 6 x 6
    c1 = self.maxPool(c1)  # 256 x 3 x 3
    c2 = self.maxPool(c2)  # 256 x 3 x 3

    



    output = torch.cat((c1,c2),1)

    output = output.view(-1,512*3*3)

    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    output = self.fc3(output)

    return output, output1, output2, output3

In [12]:
class Net1_classes(nn.Module):
  def __init__(self):
    super().__init__()
    #channel 1
    self.conv11 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv12 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv13 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv14 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv15 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    #channel 2
    self.conv21 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv22 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv23 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv24 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv25 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    self.maxPool = nn.MaxPool2d(kernel_size=(2, 2))



    self.fc1 = nn.Linear(512*3*3, 512)
    self.fc2 = nn.Linear(512, 64)
    self.fc3 = nn.Linear(64, 2)
    self.fc4 = nn.Linear(64, 10)

  def forward(self,x): 
    # spliting the channels
    c1 = torch.narrow(x,1,0,1)
    c2 = torch.narrow(x,1,1,1)

    # Channel 1
    c1 = F.relu(self.conv11(c1))  #16 x 14 x 14
    c1 = F.relu(self.conv12(c1))  # 32 x 13 x 13
    c1 = self.maxPool(c1)
    c1 = F.relu(self.conv13(c1))  # 64 x 7 x 7
    c1 = F.relu(self.conv14(c1))  # 128 x 6 x 6
    c1 = F.relu(self.conv15(c1))  # 256 x 6 x 6
    c1 = self.maxPool(c1)  # 256 x 3 x 3

    # Channel 2
    c2 = F.relu(self.conv21(c2))  # 16 x 14 x 14
    c2 = F.relu(self.conv22(c2))  # 32 x 13 x 13
    c2 = self.maxPool(c2)
    c2 = F.relu(self.conv23(c2))  # 64 x 7 x 7
    c2 = F.relu(self.conv24(c2))  # 128 x 6 x 6
    c2 = F.relu(self.conv25(c2))  # 256 x 6 x 6
    c2 = self.maxPool(c2)  # 256 x 3 x 3





    output = torch.cat((c1,c2),1)

    output = output.view(-1,512*3*3)

    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    prediction = self.fc3(output)
    aux_1 = self.fc4(output)
    aux_2 = self.fc4(output)


    return torch.cat((prediction,aux_1,aux_2),1)


In [13]:
lr, nb_epochs, batch_size = 2e-3, 200, 50
# model = Net1()
model = Net1()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()


model.to(device)
criterion.to(device)

for e in range(nb_epochs):
    for input, targets, classes in zip(train_pairs.split(batch_size), train_target.split(batch_size), train_classes.split(batch_size)):
        # output, output_aux1, output_aux2 = model(input)
        output = model(input.to(device)).to(device)
        loss = criterion(output, targets.to(device))
        # loss_aux1 = criterion(output_aux1, targets)
        # loss_aux2 = criterion(output_aux2, targets)

        # print(loss)
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    train_errors = 100 * (1 - compute_nb_errors(model, train_pairs.to(device), train_target,batch_size, False)/train_pairs.size(0))
    test_errors = 100 * (1 - compute_nb_errors(model, test_pairs.to(device), test_target,batch_size, False)/test_pairs.size(0))
    print(f"Epoch # {e+1} / Train accuracy (%): {train_errors:.2f} / Test accuracy (%): {test_errors:.2f}")

        

Epoch # 1 / Train accuracy (%): 58.70 / Test accuracy (%): 59.00
Epoch # 2 / Train accuracy (%): 61.30 / Test accuracy (%): 61.50
Epoch # 3 / Train accuracy (%): 64.00 / Test accuracy (%): 63.90
Epoch # 4 / Train accuracy (%): 64.10 / Test accuracy (%): 65.00
Epoch # 5 / Train accuracy (%): 63.90 / Test accuracy (%): 64.00
Epoch # 6 / Train accuracy (%): 62.30 / Test accuracy (%): 62.80
Epoch # 7 / Train accuracy (%): 61.20 / Test accuracy (%): 61.80
Epoch # 8 / Train accuracy (%): 60.00 / Test accuracy (%): 60.80
Epoch # 9 / Train accuracy (%): 60.20 / Test accuracy (%): 61.10
Epoch # 10 / Train accuracy (%): 61.00 / Test accuracy (%): 62.20
Epoch # 11 / Train accuracy (%): 62.80 / Test accuracy (%): 63.60
Epoch # 12 / Train accuracy (%): 64.20 / Test accuracy (%): 64.80
Epoch # 13 / Train accuracy (%): 66.10 / Test accuracy (%): 66.00
Epoch # 14 / Train accuracy (%): 67.30 / Test accuracy (%): 66.90
Epoch # 15 / Train accuracy (%): 68.80 / Test accuracy (%): 68.50
Epoch # 16 / Train 

In [14]:
lr, nb_epochs, batch_size = 1e-3, 200, 50
# model = Net1()
model = Net1_aux()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

model.to(device)
criterion.to(device)

for e in range(nb_epochs):
    for input, targets in zip(train_pairs.split(batch_size), train_target.split(batch_size)):
        # output, output_aux1, output_aux2 = model(input)
        output, output1, output2, output3 = model(input.to(device))
        loss0 = criterion(output, targets.to(device))
        loss1 = criterion(output1, targets.to(device))
        loss2 = criterion(output2, targets.to(device))
        loss3 = criterion(output3, targets.to(device))
        # loss_aux1 = criterion(output_aux1, targets)
        # loss_aux2 = criterion(output_aux2, targets)
        loss = loss0 + loss1 + loss2 + loss3
        # print(loss)
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    train_errors = 100 * (1 - compute_nb_errors(model, train_pairs.to(device), train_target,batch_size, True)/train_pairs.size(0))
    test_errors = 100 * (1 - compute_nb_errors(model, test_pairs.to(device), test_target,batch_size, True)/test_pairs.size(0))
    print(f"Epoch # {e+1} / Train accuracy (%): {train_errors:.2f} / Test accuracy (%): {test_errors:.2f}")

Epoch # 1 / Train accuracy (%): 56.20 / Test accuracy (%): 58.80
Epoch # 2 / Train accuracy (%): 62.90 / Test accuracy (%): 63.80
Epoch # 3 / Train accuracy (%): 61.30 / Test accuracy (%): 63.80
Epoch # 4 / Train accuracy (%): 70.40 / Test accuracy (%): 69.90
Epoch # 5 / Train accuracy (%): 72.40 / Test accuracy (%): 71.30
Epoch # 6 / Train accuracy (%): 73.90 / Test accuracy (%): 71.20
Epoch # 7 / Train accuracy (%): 75.10 / Test accuracy (%): 71.20
Epoch # 8 / Train accuracy (%): 71.20 / Test accuracy (%): 67.90
Epoch # 9 / Train accuracy (%): 75.70 / Test accuracy (%): 72.80
Epoch # 10 / Train accuracy (%): 76.20 / Test accuracy (%): 72.90
Epoch # 11 / Train accuracy (%): 75.70 / Test accuracy (%): 72.30
Epoch # 12 / Train accuracy (%): 75.40 / Test accuracy (%): 71.90
Epoch # 13 / Train accuracy (%): 76.50 / Test accuracy (%): 72.70
Epoch # 14 / Train accuracy (%): 76.00 / Test accuracy (%): 72.60
Epoch # 15 / Train accuracy (%): 77.60 / Test accuracy (%): 73.40
Epoch # 16 / Train 

In [83]:
lr, nb_epochs, batch_size = 1e-3, 200, 50
model1 = Net1_classes()
test_acc = training_model(model=model1, train_input=train_pairs, train_target=train_target, train_classes=train_classes, test_input = test_pairs, test_target=test_target, eta=lr, epochs=nb_epochs, batch_size=batch_size, auxiliary_loss=True)
print(test_acc)

Epoch # 1 / Train accuracy (%): 66.10 / Test accuracy (%): 62.20
Epoch # 2 / Train accuracy (%): 71.50 / Test accuracy (%): 68.70
Epoch # 3 / Train accuracy (%): 74.80 / Test accuracy (%): 72.70
Epoch # 4 / Train accuracy (%): 77.40 / Test accuracy (%): 73.90
Epoch # 5 / Train accuracy (%): 78.70 / Test accuracy (%): 74.90
Epoch # 6 / Train accuracy (%): 79.40 / Test accuracy (%): 76.40
Epoch # 7 / Train accuracy (%): 80.40 / Test accuracy (%): 77.10
Epoch # 8 / Train accuracy (%): 81.10 / Test accuracy (%): 77.30
Epoch # 9 / Train accuracy (%): 81.10 / Test accuracy (%): 77.50
Epoch # 10 / Train accuracy (%): 80.90 / Test accuracy (%): 78.00
Epoch # 11 / Train accuracy (%): 81.50 / Test accuracy (%): 78.20
Epoch # 12 / Train accuracy (%): 81.60 / Test accuracy (%): 77.90
Epoch # 13 / Train accuracy (%): 81.60 / Test accuracy (%): 77.80
Epoch # 14 / Train accuracy (%): 81.10 / Test accuracy (%): 77.70
Epoch # 15 / Train accuracy (%): 81.00 / Test accuracy (%): 77.90
Epoch # 16 / Train 

KeyboardInterrupt: 

In [62]:
lr, nb_epochs, batch_size = 2e-3, 75, 50
model1 = Net1()
test_acc = training_model(model=model1, train_input=train_pairs, train_target=train_target, train_classes=train_classes, test_input = test_pairs, test_target=test_target, eta=lr, epochs=nb_epochs, batch_size=batch_size, auxiliary_loss=False)
print(test_acc)

Epoch # 1 / Train accuracy (%): 62.30 / Test accuracy (%): 61.90
Epoch # 2 / Train accuracy (%): 67.80 / Test accuracy (%): 66.00
Epoch # 3 / Train accuracy (%): 70.20 / Test accuracy (%): 67.70
Epoch # 4 / Train accuracy (%): 72.00 / Test accuracy (%): 68.30


KeyboardInterrupt: 